Project Notebook

Author: Brenden Eum (2024)

This is the main notebook for running all code related to the project.

# Preamble

In [12]:
cd("/Users/brenden/Desktop/ddm-sampling-weights")
import Pkg;
Pkg.add("RCall");
using RCall;

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


---

# Preprocessing

**CLEAN:**

Generate variables and transform the data into something that looks like the final dataset.

**FILTER:**

Subjects are run through 1 trial-level filter and 3 participant-level filters.

At the *trial-level*, we drop any trial that is faster than .325 seconds. We selected this cutoff based on the average RT of the worst-performing participant under time pressure from Milosavljevic et al. (2010), who had an accuracy rate of 63.5%.

At the *participant-level*, we drop any participants who:
1. [Missing] Are missing data on more than 95% of trials. Missing can occur if participant [1] took too long (>12s) or [2] decided too quickly (<.325s).
2. [Accuracy] Do not perform better than chance on the trials that they did respond.
3. [Speed] Answered too quickly on average. Their average RT must be longer than .350 seconds, which is the average of the mean non-decision time estimated in Milosavljevic et al. (2010).

**SPLIT:**

Split the data in exploratory, confirmatory, and joint datasets.

| IN | OUT |
| --- | --- |
| data/raw_data/* | data/processed_data/* |
| | analysis/output/text/preprocessing/* |

In [26]:
R"source('analysis/preprocessing/clean_data.R')"
R"source('analysis/preprocessing/filter_data.R')"
R"source('analysis/preprocessing/split_data.R')"
R"source('analysis/preprocessing/preprocessing_analysis.R')"

[1] "[NAs introduced by coercion] error is ok!"


┌ Warning: RCall.jl: Warning in eval(ei, envir) : NAs introduced by coercion
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: Warning in eval(ei, envir) :
│   At least 1 trial failed the min(RT) filter.
│ Warning in eval(ei, envir) :
│   At least 1 subject failed the non-missing filter.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{VecSxp}
$value
NULL

$visible
[1] FALSE



**NOTES**

59 subjects collected. 52 subjects passed the filters. 26 subjects to exploratory, 26 to confirmatory. Trials dropped $\in [0,14]$, with $mean=.96$.

---
# Basic Psychometrics

Choice as a function of slot machine mean. Response time as a function of slot machine mean. Number of samples as a function of slot machine mean (this should look a lot like RT).

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/BasicPsychometrics_* |

In [29]:
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_Choice.R')"
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_RT.R')"
R"source('analysis/helpers/model_free_analysis/BasicPsychometrics_NumSamples.R')"

┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{VecSxp}
$value
[1] "analysis/output/figures//BasicPsycho_NumSamples.pdf"

$visible
[1] FALSE



**NOTES**

Psychometric curve shows evidence of loss aversion. When slot machine mean is 0, people select skip more than they select play. Response times (and therefore number of samples) is highest when slot machine mean is close to 0 (i.e. when it's the most difficult).

---
# Choice Process

How is the choice process affected by features of the experiment?

First I plot RT as a function of the first sample. I truncate first sample to [-7, 7] since there's hardly any data outside these bounds (makes sense since most extreme slot machines are $\pm 2$ with $SD=2$.)

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/ChoiceProcess_* |

In [35]:
R"source('analysis/helpers/model_free_analysis/ChoiceProcess_RT_FirstSample.R')"

┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172


RObject{VecSxp}
$value
[1] "analysis/output/figures//ChoiceProcess_RT_FirstSample.pdf"

$visible
[1] FALSE



**NOTES**

Ignoring extreme first samples, participants are typically faster to respond the larger the magnitude of the sample. This is consistent with the DDM. For extremely low first samples, participatns are even faster to respond, still consistent with the DDM. However, for extremely high first samples, participants are significantly slower to respond, which is inconsistent with the DDM. This seems like a "too good to be true" effect. This is not just due to fewer subjects at the extremes. I checked and most all subjects are seeing $\pm 7$. SEs are wider at the extremes, hinting at more heterogeneous behavior, but it's significant still.

When it comes to extremely positive first samples, I'd be willing to bet that people are either savoring the good information, or they're taking more caution to make sure they're making a good choice. Is it possible that decisions boundaries are adjusted after seeing the first sample? The equivalent process can be handled by a dynamic weight on evidence that adjusts thorughout the course of a decision. This dynamic weight would compress extremely large positive first samples. It could also do other things like change the weight on late vs. early evidence (see Choice Biases).

---
# Choice Biases

Plot psychometric curves, with a different line for each level of sample. Samples are binned with widths of 2: [-5, -3), [-3, -1), [-1, 1), [1, 3), [3,5).

We look at the influence of the first sample, middle samples, the last sample (which is typically only viewed for a blink of an eye due to the stimulus refresh rate), early samples (2, 3, 4), and late samples (-4, -3, -2). Note that when RTs are less than 1.8 seconds, these early and late samples can overlap. For instance, if RT is 1.8 s, early sample 4 might be the same as late sample -4.

| IN | OUT |
| --- | --- |
| data/processed_data/* | analysis/output/figures/ChoiceBiases_* |

In [34]:
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_First.R')"
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Middle.R')"
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Last.R')"
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Early.R')"
R"source('analysis/helpers/model_free_analysis/ChoiceBiases_Late.R')"

┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode', 'slot_mean'. You can override
│ using the `.groups` argument.
│ `summarise()` has grouped output by 'slot_mean'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode', 'slot_mean'. You can override
│ using the `.groups` argument.
│ `summarise()` has grouped output by 'slot_mean'. You can override using the
│ `.groups` argument.
│ Coordinate system already present. Adding new coordinate system, which will
│ replace the existing one.
└ @ RCall /Users/brenden/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: `summarise()` has grouped output by 'parcode', 'slot_mean'. You can override
│ using the `.groups` argument.
│ `summarise()` has grouped output by 'slot_mean'. You can overr

RObject{VecSxp}
$value
[1] "analysis/output/figures//ChoiceBiases_LateSampleBias.pdf"

$visible
[1] FALSE



**NOTES**

There is noticeable separation in the lines for all but last samples, suggesting that participants are integrating this evidence. Separation in first sample is larger than for middle samples, suggesting that the first sample might be weighted more than following samples. However, when we look at samples 2, 3, and 4, the separation is smaller than for samples -4, -3, -2 (indexed from the last sample, -1). I think this would mean that late samples are weighted more than early samples.